## Intro

This python script will do as follow 

1. Genereate dataset of 100 sentenses 
2. Pick the words of prefrence in the dataset 

The Machine learning part 
* show and proof the more distinct prefrencese are the more accurate your classifier is 

In [3]:
# Code to generate Sentenses with verb_list
import random
import csv

# Can
verb_list =  ["run","just","do","it","let's","make","America","Great","Again","I","Doooiii"]
def randome_gen(list_of_words):
    number_of_words = len(verb_list) # Get the length of the word list
    random_word_number = random.randint(0,number_of_words -1) # pick a random number up to the end of the list
    selected_word = list_of_words[random_word_number] # select the word at the number spot
    return selected_word

# The <Adjective> <Noun> <verb> the <adjective> <noun>


In [5]:
# Test The Method 
list_of_words = ' '
for x in range(0,10):
    list_of_words += " " + randome_gen(verb_list)
print(list_of_words)


  do Great Doooiii run run I America America do I


## Machine Learning part 1) Generate

In [6]:
# Author Mr.Green
verb_list_G = ["I","run","let's","Do","it","Good","Morning","saw","cow","mac","python","google"
             ,"like","love","am","one","page","knock","show","jump","we"]
#Author Mr.Blue
verb_list_B = ["I","love","python","google","car","speed","just","do","it",
               "meet","package","sound","smile","eat","done","sleep","walk","talk","hack","crack","track","stack"]


In [14]:
sentences_G = ' '
sentences_B = ' '
for x in range(0,1000):
    sentences_G += ' ' + randome_gen(verb_list_G)
    sentences_B += ' ' + randome_gen(verb_list_B)

span = 10
words_G = sentences_G.split(' ')
words_B = sentences_B.split(' ')

words_list_G = [' '.join(words_G[i:i+span]) for i in range(0, len(words_G), span)]
words_list_B = [' '.join(words_B[i:i+span]) for i in range(0, len(words_B), span)]

filepath = '/home/rwx/Documents/Fontys-assignment/ads/ML/ml06-NaiveBayes/my_corpus.csv' 
def write_to_csv(mylist):
    try:
        writer = csv.writer(open(filepath,'a',))
        writer.writerows(mylist)
    except:
        print('something happened')



In [47]:
# Write to file 
write_to_csv(words_list_G)
write_to_csv(words_list_B)
# Spliting and combining 
words_list = words_list_B + words_list_G

# Authors of each line
authors = []
for x in range(0,101):
    authors.append('mr.Green')
for x in range(0,101):
    authors.append('mr.Blue')

In [48]:
# This step we define the cross validation 
from sklearn.cross_validation import cross_val_score, KFold
from numpy import mean
from scipy.stats import sem

def evaluate_cross_validation(clf, X, y, K):
    # Create a k-fold cross validation iterator of k=5 folds
    cv = KFold(len(y), K, shuffle=True, random_state=0)
    # By default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print(scores)
    print("Mean score: {0:.3f}".format(mean(scores)))
    print("Standard error of the mean: (+/-{0:.3f})".format(sem(scores)))


In [49]:
# Note that we create our validation folds from our complete dataset as contrasted to creating folds
# from the training set in the SVM notebook
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

print()
print('Mr:Green')
print(vec.fit_transform(words_list).toarray()[:2])
print()
print(vec.get_feature_names())


Mr:Green
[[1 0 2 0 0 0 1 0 1 0 0 0 0 2 0 0 0]
 [2 0 1 0 1 0 1 0 1 0 1 0 1 1 0 0 0]]

['car', 'cow', 'do', 'good', 'google', 'it', 'just', 'let', 'love', 'mac', 'meet', 'morning', 'package', 'python', 'run', 'saw', 'speed']


In [51]:
'''
Naive Bayes classifier (see below) can only deal with numeric data.
So we have to map the texts to numeric data. That's in short what the vectorizer does: 
it creates a vector of features that we can give numeric values. The CountVectorizer is one of the simplest
vectorizers available: it just creates a feature for each unique word in the text an then counts occurrences of
each word in a text.

There are also other vectorizers available in sklearn, such as the HashingVectorizer.
Using the HashingVectorizer leads to a smaller feature space as different unique words may be hashed to the same bucket.
The buckets form the feature space. Also, vectorizers may have paramaters. See the sklearn docs for which params you can use.
'''
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])


evaluate_cross_validation(clf, words_list,authors, 5)

[ 1.  1.  1.  1.  1.]
Mean score: 1.000
Standard error of the mean: (+/-0.000)


In [52]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(words_list, authors, test_size=0.25, random_state=33)
print(X_train[:2])
print(y_train[:2])

['it Do Do saw run cow cow Do Morning python', 'mac Good I run python I mac Good Do Do']
['mr.Blue', 'mr.Blue']


In [53]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print("Accuracy on training set:")
    print(clf.score(X_train, y_train))
    print("Accuracy on testing set:")
    print(clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print("Classification Report:")
    print(metrics.classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(y_test, y_pred))

In [54]:
train_and_evaluate(clf, X_train, X_test, y_train, y_test)

Accuracy on training set:
1.0
Accuracy on testing set:
1.0
Classification Report:
             precision    recall  f1-score   support

    mr.Blue       1.00      1.00      1.00        26
   mr.Green       1.00      1.00      1.00        25

avg / total       1.00      1.00      1.00        51

Confusion Matrix:
[[26  0]
 [ 0 25]]


In [55]:
clf.predict(["python mac just do it amaerica hey"])

array(['mr.Blue'], 
      dtype='<U8')